# PROJET BI423 

In [1]:
import matplotlib.pyplot as plt


Vérif séquence d'ADN valide

In [4]:
def seq_init():
    global seq_plasmide
    seq_plasmide = input("Entrer une séquence d'ADN valide: ").upper()
    for nucléotide in seq_plasmide:
        if nucléotide not in 'ATGC':
            print("Erreur: La séquence contient des caractères non valides.")
            return 
    print("La séquence est valide.")
    return seq_plasmide

seq_init()


La séquence est valide.


'GATTACCTGCGGCAAAACATTGCGCAACAGCATGATCTCGACTCGCTGGCGCAGCGAGTAATGATGAGTCGCCGCACATTAACTCGCCATTTTATGAAAGCGAC'

caclul ratio

In [5]:
def trouver_point_inversion(seq_plasmide, taille_fenetre=1):
    ratios = []
    points_inversion = []
    positions = range(len(seq_plasmide) - taille_fenetre + 1)
    
    for i in positions:
        fenetre = seq_plasmide[i:i+taille_fenetre]
        nbC = fenetre.count('C')
        nbG = fenetre.count('G')
        total = nbC + nbG
        if total > 0:
            ratio = (nbC - nbG) / total
        else:
            ratio = 0
        ratios.append(ratio)


    for i in range(1, len(ratios)):
        if (ratios[i-1] < 0 and ratios[i] >= 0 and ratios[i+1] > 0) or (ratios[i-1] > 0 and ratios[i] <= 0 and ratios[i+1] < 0):
            print(f"Point d'inversion trouvé à la position {i}")
            points_inversion.append(i)
    

    plt.figure(figsize=(10, 6))
    plt.plot(positions, ratios, label='Ratio (nbC-nbG)/(nbC+nbG) par fenêtre')
    plt.axhline(0, color='red', linestyle='--')


    for point in points_inversion:
        plt.scatter(point, 0, s=100, color='yellow', edgecolors='black', zorder=10)

    plt.xlabel('Position')
    plt.ylabel('Ratio (nbC-nbG)/(nbC+nbG)')
    plt.title('Recherche de point d\'inversion dans la séquence d\'ADN')
    plt.legend()
    plt.show()


trouver_point_inversion(seq_plasmide, taille_fenetre=4)


Point d'inversion trouvé à la position 1
Point d'inversion trouvé à la position 7
Point d'inversion trouvé à la position 11
Point d'inversion trouvé à la position 21
Point d'inversion trouvé à la position 33
Point d'inversion trouvé à la position 44
Point d'inversion trouvé à la position 48
Point d'inversion trouvé à la position 52
Point d'inversion trouvé à la position 68


IndexError: list index out of range

calcul pourcentage GC 